## About this notebook
- Joinging dataframe
- Join integrity + missingness report
- Confirming target distribution + leakage checks (drop unsafe columns)
- Quick no-show rate cuts (day_of_week / specialty / insurance / clinic) to guide next steps

#### Import Libraries and display settings

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

#### Defining Data Path

In [2]:
DATA_DIR = Path("../data/")  

files = {
    "fact": DATA_DIR / "fact_appointments.csv",
    "patients": DATA_DIR / "dim_patients.csv",
    "providers": DATA_DIR / "dim_providers.csv",
    "clinics": DATA_DIR / "dim_clinics.csv",
    "dates": DATA_DIR / "dim_dates.csv",
}

files

{'fact': WindowsPath('../data/fact_appointments.csv'),
 'patients': WindowsPath('../data/dim_patients.csv'),
 'providers': WindowsPath('../data/dim_providers.csv'),
 'clinics': WindowsPath('../data/dim_clinics.csv'),
 'dates': WindowsPath('../data/dim_dates.csv')}

#### Load Data

In [3]:
fact = pd.read_csv(files["fact"])
patients = pd.read_csv(files["patients"])
providers = pd.read_csv(files["providers"])
clinics = pd.read_csv(files["clinics"])
dates = pd.read_csv(files["dates"])

{
    "fact": fact.shape,
    "patients": patients.shape,
    "providers": providers.shape,
    "clinics": clinics.shape,
    "dates": dates.shape
}

{'fact': (120000, 10),
 'patients': (5000, 3),
 'providers': (220, 4),
 'clinics': (120000, 5),
 'dates': (1096, 5)}

##### Clinic table is not a true dimension table yet. It’s clinic info repeated per appointment.

#### Clean column names

In [4]:
def clean_cols(df):
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

fact = clean_cols(fact)
patients = clean_cols(patients)
providers = clean_cols(providers)
clinics = clean_cols(clinics)
dates = clean_cols(dates)

print("fact:", fact.columns.tolist())
print("patients:", patients.columns.tolist())
print("providers:", providers.columns.tolist())
print("clinics:", clinics.columns.tolist())
print("dates:", dates.columns.tolist())

fact: ['appointment_id', 'patient_id', 'clinic_id', 'provider_id', 'date_id', 'appointment_time', 'appointment_date', 'lead_time_days', 'wait_time_minutes', 'is_no_show']
patients: ['patient_id', 'insurance_type', 'age']
providers: ['provider_id', 'specialty', 'clinic_id', 'clinic_assignment']
clinics: ['clinic_id', 'clinic_assignment', 'clinic_name', 'city', 'hours']
dates: ['date_id', 'day_of_week', 'appointment_date', 'month_name', 'year']


#### Building True dim_clinic

In [5]:
dim_clinics = clinics.drop_duplicates(subset=["clinic_id"]).copy()

print("clinics raw shape:", clinics.shape)
print("dim_clinics shape:", dim_clinics.shape)
dim_clinics.head()

clinics raw shape: (120000, 5)
dim_clinics shape: (15, 5)


,clinic_id,clinic_assignment,clinic_name,city,hours
0,C007,Full-time,Oncology Clinic,Chicago,10:00-19:00
1,C002,Full-time,Cardiology Clinic,Los Angeles,07:00-15:00
2,C012,Full-time,Psychiatry Clinic,Atlanta,08:00-18:00
4,C015,Full-time,Pulmonology Clinic,Houston,08:00-16:00
5,C013,Full-time,Ophthalmology Clinic,Houston,08:00-18:00


#### Leakage Confirmation - No show and wait time

In [6]:
if "wait_time_minutes" in fact.columns:
    leak_check = (
        fact.assign(wait_missing=fact["wait_time_minutes"].isna())
            .groupby(["is_no_show", "wait_missing"])
            .size()
            .unstack(fill_value=0)
    )
    display(leak_check)

    # DROP leakage column
    fact = fact.drop(columns=["wait_time_minutes"])
    print("Dropped wait_time_minutes ✅")
else:
    print("No wait_time_minutes found ✅")

wait_missing,False,True
is_no_show,,
0,98656,0
1,0,21344


Dropped wait_time_minutes ✅


In [7]:
providers_for_join = providers.drop(columns=["clinic_id"], errors="ignore")
providers_for_join.columns.tolist()

['provider_id', 'specialty', 'clinic_assignment']

#### Joining the tables

In [8]:
df = fact.merge(patients, on="patient_id", how="left", validate="many_to_one")
df = df.merge(providers_for_join, on="provider_id", how="left", validate="many_to_one")
df = df.merge(dim_clinics, on="clinic_id", how="left", validate="many_to_one")
df = df.merge(dates, on="date_id", how="left", validate="many_to_one")

print("df shape:", df.shape)
df.head(3)

df shape: (120000, 21)


,appointment_id,patient_id,clinic_id,provider_id,date_id,appointment_time,appointment_date_x,lead_time_days,is_no_show,insurance_type,age,specialty,clinic_assignment_x,clinic_assignment_y,clinic_name,city,hours,day_of_week,appointment_date_y,month_name,year
0,A0000001,P004264,C007,PR00120,1,9:00:00 AM,2023-06-11,5,0,Public,67,Oncology,Full-time,Full-time,Oncology Clinic,Chicago,10:00-19:00,Sun,2023-06-11,Jun,2023.0
1,A0000002,P002201,C002,PR00045,2,3:30:00 PM,2023-05-03,4,0,Private,12,Cardiology,Full-time,Full-time,Cardiology Clinic,Los Angeles,07:00-15:00,Thu,2024-07-04,Jul,2024.0
2,A0000003,P002984,C012,PR00105,3,4:45:00 PM,2024-07-09,7,0,Self-pay,51,Psychiatry,Full-time,Full-time,Psychiatry Clinic,Atlanta,08:00-18:00,Wed,2025-09-17,Sep,2025.0


In [10]:
df.columns

Index(['appointment_id', 'patient_id', 'clinic_id', 'provider_id', 'date_id', 'appointment_time', 'appointment_date_x',
       'lead_time_days', 'is_no_show', 'insurance_type', 'age', 'specialty', 'clinic_assignment_x',
       'clinic_assignment_y', 'clinic_name', 'city', 'hours', 'day_of_week', 'appointment_date_y', 'month_name',
       'year'],
      dtype='object')

In [11]:
# 1) Fix appointment_date duplicates
# Keep the appointment date from fact (x) and drop dates copy (y)
if "appointment_date_x" in df.columns:
    df = df.rename(columns={"appointment_date_x": "appointment_date"})
if "appointment_date_y" in df.columns:
    df = df.drop(columns=["appointment_date_y"])

# 2) Fix clinic_assignment duplicates (rename both for clarity)
rename_map = {}
if "clinic_assignment_x" in df.columns:
    rename_map["clinic_assignment_x"] = "provider_clinic_assignment"
if "clinic_assignment_y" in df.columns:
    rename_map["clinic_assignment_y"] = "clinic_staffing_type"

df = df.rename(columns=rename_map)

# Quick check: show columns involved
cols_to_check = [c for c in df.columns if "appointment_date" in c or "clinic_assignment" in c]
print("Cleaned columns:", cols_to_check)

df.head(3)

Cleaned columns: ['appointment_date', 'provider_clinic_assignment']


,appointment_id,patient_id,clinic_id,provider_id,date_id,appointment_time,appointment_date,lead_time_days,is_no_show,insurance_type,age,specialty,provider_clinic_assignment,clinic_staffing_type,clinic_name,city,hours,day_of_week,month_name,year
0,A0000001,P004264,C007,PR00120,1,9:00:00 AM,2023-06-11,5,0,Public,67,Oncology,Full-time,Full-time,Oncology Clinic,Chicago,10:00-19:00,Sun,Jun,2023.0
1,A0000002,P002201,C002,PR00045,2,3:30:00 PM,2023-05-03,4,0,Private,12,Cardiology,Full-time,Full-time,Cardiology Clinic,Los Angeles,07:00-15:00,Thu,Jul,2024.0
2,A0000003,P002984,C012,PR00105,3,4:45:00 PM,2024-07-09,7,0,Self-pay,51,Psychiatry,Full-time,Full-time,Psychiatry Clinic,Atlanta,08:00-18:00,Wed,Sep,2025.0


In [13]:
df.columns

Index(['appointment_id', 'patient_id', 'clinic_id', 'provider_id', 'date_id', 'appointment_time', 'appointment_date',
       'lead_time_days', 'is_no_show', 'insurance_type', 'age', 'specialty', 'provider_clinic_assignment',
       'clinic_staffing_type', 'clinic_name', 'city', 'hours', 'day_of_week', 'month_name', 'year'],
      dtype='object')

In [12]:
missing = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
display(missing[missing > 0].head(30))

print(df["is_no_show"].value_counts())
print(df["is_no_show"].value_counts(normalize=True).round(4))

day_of_week    99.3
month_name     99.3
year           99.3
dtype: float64

is_no_show
0    98656
1    21344
Name: count, dtype: int64
is_no_show
0    0.8221
1    0.1779
Name: proportion, dtype: float64


In [14]:
df = df.drop(columns=["day_of_week", "month_name", "year"], errors="ignore")

In [15]:
df = fact.merge(patients, on="patient_id", how="left", validate="many_to_one")
df = df.merge(providers_for_join, on="provider_id", how="left", validate="many_to_one")
df = df.merge(dim_clinics, on="clinic_id", how="left", validate="many_to_one")

In [18]:
fact["appointment_date"] = pd.to_datetime(fact["appointment_date"])
dates["appointment_date"] = pd.to_datetime(dates["appointment_date"])

In [19]:
print("df appointment_date dtype:", df["appointment_date"].dtype)
print("dates appointment_date dtype:", dates["appointment_date"].dtype)

df["appointment_date"].head(), dates["appointment_date"].head()

df appointment_date dtype: object
dates appointment_date dtype: datetime64[ns]


(0    2023-06-11
 1    2023-05-03
 2    2024-07-09
 3    2024-07-07
 4    2024-07-17
 Name: appointment_date, dtype: object,
 0   2023-06-11
 1   2024-07-04
 2   2025-09-17
 3   2025-03-20
 4   2025-05-01
 Name: appointment_date, dtype: datetime64[ns])

In [20]:
df["appointment_date"] = pd.to_datetime(df["appointment_date"], errors="coerce")
dates["appointment_date"] = pd.to_datetime(dates["appointment_date"], errors="coerce")

print("After conversion:")
print("df appointment_date dtype:", df["appointment_date"].dtype)
print("dates appointment_date dtype:", dates["appointment_date"].dtype)
print("df null dates:", df["appointment_date"].isna().sum())
print("dates null dates:", dates["appointment_date"].isna().sum())

After conversion:
df appointment_date dtype: datetime64[ns]
dates appointment_date dtype: datetime64[ns]
df null dates: 0
dates null dates: 0


In [21]:
dup_dates = dates["appointment_date"].duplicated().sum()
print("Duplicate appointment_date rows in dates:", dup_dates)

Duplicate appointment_date rows in dates: 0


In [22]:
df = df.merge(
    dates,
    on="appointment_date",
    how="left",
    validate="many_to_one"
)

print("df shape:", df.shape)

df shape: (120000, 21)


In [23]:
missing = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
display(missing[missing > 0].head(20))

Series([], dtype: float64)

In [25]:
df.head()

,appointment_id,patient_id,clinic_id,provider_id,date_id_x,appointment_time,appointment_date,lead_time_days,is_no_show,insurance_type,age,specialty,clinic_assignment_x,clinic_assignment_y,clinic_name,city,hours,date_id_y,day_of_week,month_name,year
0,A0000001,P004264,C007,PR00120,1,9:00:00 AM,2023-06-11,5,0,Public,67,Oncology,Full-time,Full-time,Oncology Clinic,Chicago,10:00-19:00,1,Sun,Jun,2023
1,A0000002,P002201,C002,PR00045,2,3:30:00 PM,2023-05-03,4,0,Private,12,Cardiology,Full-time,Full-time,Cardiology Clinic,Los Angeles,07:00-15:00,201,Wed,May,2023
2,A0000003,P002984,C012,PR00105,3,4:45:00 PM,2024-07-09,7,0,Self-pay,51,Psychiatry,Full-time,Full-time,Psychiatry Clinic,Atlanta,08:00-18:00,334,Tue,Jul,2024
3,A0000004,P002734,C012,PR00085,4,3:45:00 PM,2024-07-07,17,0,Private,31,Psychiatry,Full-time,Full-time,Psychiatry Clinic,Atlanta,08:00-18:00,883,Sun,Jul,2024
4,A0000005,P002814,C015,PR00189,5,5:15:00 PM,2024-07-17,24,0,Private,3,Pulmonology,Full-time,Full-time,Pulmonology Clinic,Houston,08:00-16:00,202,Wed,Jul,2024


#### Convert appointment_time and extract hour/minute

In [27]:
# Parse as time (12-hour with AM/PM). Adjust if your raw format differs.
df["appointment_time_parsed"] = pd.to_datetime(
    df["appointment_time"].astype(str),
    format="%I:%M:%S %p",
    errors="coerce"
)

print("parsed nulls:", df["appointment_time_parsed"].isna().sum())

df["appt_hour"] = df["appointment_time_parsed"].dt.hour
df["appt_minute"] = df["appointment_time_parsed"].dt.minute

parsed nulls: 120000


In [28]:
df["appointment_time"].astype(str).unique()[:10]

array(['2026-02-23 09:00:00', '2026-02-23 15:30:00',
       '2026-02-23 16:45:00', '2026-02-23 15:45:00',
       '2026-02-23 17:15:00', '2026-02-23 08:00:00',
       '2026-02-23 10:15:00', '2026-02-23 15:00:00',
       '2026-02-23 19:00:00', '2026-02-23 13:00:00'], dtype=object)

In [29]:
# confirm dtype (should be datetime64[ns])
print(df["appointment_time"].dtype)

# extract features
df["appt_hour"] = df["appointment_time"].dt.hour
df["appt_minute"] = df["appointment_time"].dt.minute

df[["appointment_time", "appt_hour", "appt_minute"]].head()

datetime64[ns]


,appointment_time,appt_hour,appt_minute
0,2026-02-23 09:00:00,9,0
1,2026-02-23 15:30:00,15,30
2,2026-02-23 16:45:00,16,45
3,2026-02-23 15:45:00,15,45
4,2026-02-23 17:15:00,17,15


In [30]:
df = df.drop(columns=["appointment_time_parsed"], errors="ignore")

In [31]:
def no_show_rate(col, top=15):
    return df.groupby(col)["is_no_show"].mean().sort_values(ascending=False).head(top)

display(no_show_rate("day_of_week", top=10))
display(no_show_rate("insurance_type", top=10))
display(no_show_rate("specialty", top=15))

day_of_week
Wed    0.181091
Thu    0.180692
Fri    0.180143
Tue    0.177448
Sun    0.177389
Sat    0.176053
Mon    0.172356
Name: is_no_show, dtype: float64

insurance_type
Public      0.178255
Private     0.177727
Self-pay    0.177266
Name: is_no_show, dtype: float64

specialty
Dentistry                  0.186347
Orthopedics                0.181353
Obstetrics & Gynecology    0.181256
Gastroenterology           0.180993
Urology                    0.178213
Psychiatry                 0.178108
Ophthalmology              0.177879
Dermatology                0.176534
Cardiology                 0.176274
Otolaryngology             0.175547
Pulmonology                0.174128
Neurology                  0.174052
Oncology                   0.172820
Endocrinology              0.170374
Family Medicine            0.168917
Name: is_no_show, dtype: float64

# Exploratory Data Analysis 

## 1️⃣ No-Show Rate by Day of Week

Observed range:
- Lowest ≈ 17.2%
- Highest ≈ 18.1%

The variation across days of the week is small (~1% difference).

**Interpretation:**
- Day of week has some signal.
- However, it is likely a weak standalone predictor.
- It may still contribute marginal improvement in a multivariate model.

---

## 2️⃣ No-Show Rate by Insurance Type

Observed range:
- Public ≈ 17.83%
- Private ≈ 17.77%
- Self-pay ≈ 17.73%

The difference across insurance types is minimal (< 0.1%).

**Interpretation:**
- Insurance type appears to have very weak predictive power on its own.
- It may still provide minor interaction effects in a full model.

---

## 3️⃣ No-Show Rate by Specialty

Observed range:
- Highest: Dentistry ≈ 18.63%
- Lowest: Family Medicine ≈ 16.89%

Spread ≈ 1.7%.

**Interpretation:**
- Specialty shows moderate variation.
- This suggests some specialties experience slightly higher no-show behavior.
- This feature may contribute useful signal in modeling.

---

# 🎯 Key Takeaway

So far, these features represent **group-level averages** and show only modest variation.

Historically in no-show prediction problems, the strongest predictors are:

1. Patient historical behavior  
2. Lead time (already present in dataset)  
3. First-appointment indicator  

Therefore, the next step is to engineer patient-level historical features in a time-safe (no-leakage) manner.

#### Sorting Appointments

In [32]:
df = df.sort_values(
    ["patient_id", "appointment_date", "appointment_time"]
).reset_index(drop=True)

df[["patient_id", "appointment_date", "appointment_time"]].head(10)

,patient_id,appointment_date,appointment_time
0,P000001,2023-01-19,2026-02-23 12:15:00
1,P000001,2023-01-20,2026-02-23 14:30:00
2,P000001,2023-04-17,2026-02-23 18:15:00
3,P000001,2023-05-06,2026-02-23 07:00:00
4,P000001,2023-05-08,2026-02-23 18:45:00
5,P000001,2023-05-22,2026-02-23 14:45:00
6,P000001,2023-08-31,2026-02-23 08:00:00
7,P000001,2023-11-18,2026-02-23 10:00:00
8,P000001,2024-03-16,2026-02-23 08:00:00
9,P000001,2024-08-19,2026-02-23 08:15:00


#### Creating Historical Features

In [33]:
# Number of previous appointments
df["prev_appointments"] = df.groupby("patient_id").cumcount()

# Previous no-shows (shift to avoid leakage)
df["prev_no_shows"] = (
    df.groupby("patient_id")["is_no_show"]
      .cumsum()
      .shift(1)
      .fillna(0)
)

# Historical no-show rate
df["hist_no_show_rate"] = (
    df["prev_no_shows"] / df["prev_appointments"]
).replace([np.inf, np.nan], 0)

# First appointment flag
df["is_first_appointment"] = (df["prev_appointments"] == 0).astype(int)

df[[
    "patient_id",
    "appointment_date",
    "is_no_show",
    "prev_appointments",
    "prev_no_shows",
    "hist_no_show_rate",
    "is_first_appointment"
]].head(20)

,patient_id,appointment_date,is_no_show,prev_appointments,prev_no_shows,hist_no_show_rate,is_first_appointment
0,P000001,2023-01-19,0,0,0.0,0.0,1
1,P000001,2023-01-20,0,1,0.0,0.0,0
2,P000001,2023-04-17,0,2,0.0,0.0,0
3,P000001,2023-05-06,0,3,0.0,0.0,0
4,P000001,2023-05-08,0,4,0.0,0.0,0
5,P000001,2023-05-22,0,5,0.0,0.0,0
6,P000001,2023-08-31,0,6,0.0,0.0,0
7,P000001,2023-11-18,0,7,0.0,0.0,0
8,P000001,2024-03-16,0,8,0.0,0.0,0
9,P000001,2024-08-19,0,9,0.0,0.0,0


In [34]:
df.groupby(pd.qcut(df["hist_no_show_rate"], q=5, duplicates="drop"))["is_no_show"].mean()

C:\Users\icons\AppData\Local\Temp\ipykernel_18356\2934960968.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.qcut(df["hist_no_show_rate"], q=5, duplicates="drop"))["is_no_show"].mean()


hist_no_show_rate
(-0.001, 0.125]    0.173683
(0.125, 0.19]      0.175902
(0.19, 0.273]      0.184143
(0.273, 1.0]       0.182206
Name: is_no_show, dtype: float64

#### Checking distribution validation

In [35]:
df["prev_appointments"].value_counts().head(10)

prev_appointments
0    5000
1    5000
2    5000
3    5000
4    5000
5    5000
6    5000
7    5000
8    5000
9    4996
Name: count, dtype: int64

In [36]:
df.groupby("patient_id").size().describe()

count    5000.000000
mean       24.000000
std         4.888392
min         9.000000
25%        21.000000
50%        24.000000
75%        27.000000
max        46.000000
dtype: float64

#### Checking if patient has showed before

In [37]:
df["has_prev_no_show"] = (df["prev_no_shows"] > 0).astype(int)

df.groupby("has_prev_no_show")["is_no_show"].mean()

has_prev_no_show
0    0.173339
1    0.178951
Name: is_no_show, dtype: float64

#### Checking last appointment

In [38]:
df["last_no_show"] = (
    df.groupby("patient_id")["is_no_show"]
      .shift(1)
      .fillna(0)
)

df.groupby("last_no_show")["is_no_show"].mean()

last_no_show
0.0    0.178325
1.0    0.175632
Name: is_no_show, dtype: float64

#### Checking lead time (days)

In [39]:
df.groupby(pd.qcut(df["lead_time_days"], q=5, duplicates="drop"))["is_no_show"].mean()

C:\Users\icons\AppData\Local\Temp\ipykernel_18356\2707207744.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.qcut(df["lead_time_days"], q=5, duplicates="drop"))["is_no_show"].mean()


lead_time_days
(-0.001, 5.0]    0.075680
(5.0, 11.0]      0.136825
(11.0, 16.0]     0.179317
(16.0, 22.0]     0.219037
(22.0, 63.0]     0.294310
Name: is_no_show, dtype: float64

# Lead Time Impact on No-Show Rate

Lead time shows a strong monotonic relationship with no-show probability.

| Lead Time (Days) | No-Show Rate |
|------------------|-------------|
| 0–5              | 7.6%        |
| 5–11             | 13.7%       |
| 11–16            | 17.9%       |
| 16–22            | 21.9%       |
| 22–63            | 29.4%       |

## Key Insight

As the number of days between scheduling and appointment increases, the likelihood of no-show increases significantly.

This is the strongest predictor identified so far.

Lead time should be treated as a core modeling feature.

### Removing unwanted columns

In [41]:
# Drop raw IDs that are not useful for EDA
df_eda = df.drop(columns=[
    "appointment_id",
    "patient_id",
    "provider_id",
    "date_id"
], errors="ignore").copy()

df_eda.shape

(120000, 26)

#### Save as CSV

In [42]:
from pathlib import Path

OUTPUT_DIR = Path("../data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / "appointments_modeling_base.csv"

df_eda.to_csv(output_path, index=False)

print("Saved to:", output_path)

Saved to: ..\data\processed\appointments_modeling_base.csv


#### Saving as Parquet

In [43]:
parquet_path = OUTPUT_DIR / "appointments_modeling_base.parquet"
df_eda.to_parquet(parquet_path, index=False)

print("Saved parquet to:", parquet_path)

Saved parquet to: ..\data\processed\appointments_modeling_base.parquet
